## <font color="#521921"> APVC: *Challenge 2*
##### Vera Cruz, Dilan.

> <font color="##194852">The objective of this exercise is to develop a Python script that reads images containing several coins and 
and calculates an estimate for the number of coins present in the image.

The approach used was:
- Binarizing the images (with Otsu method or use fixed thresholds) 
- Morphological operations in order to: 
    
    1) reduce the noise in the binarized image; 
    
    2) separate the regions corresponding to the coins

</font>


In [225]:
!pip install opencv-python

In [226]:
import cv2
import numpy as np
import os

# Introduction
The appoach used in this assignment was to perform the image processing and the A/B testing in one or two coin images and then proceed to use the same methods in the rest of the images. 

## Show the images

In [227]:
# Change the directory here to the folder where the images are located in your machine.
directory = r'C:\Users\User\Downloads\Desafio2 - contador de moedas-20231006\Challenge2'

In [228]:
for filename in os.listdir(directory):
    if filename.endswith(".jpg"):
        image_path = os.path.join(directory, filename)

        image = cv2.imread(image_path)

        if image is not None:
            #cv2.imshow('Image', image) # Remove the comment here to display all the images
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print(f"Failed to load image from {image_path}")


As it can be seen here there are various coins images with different characteristics. Some of them are brighter than other, the sizes are different, so is the shape and some of them have even holes in them. In order to accurately get the right number of coins it is needed to first do a image pre-processing.

## Normalize the image

Normalization is being used to standardize the pixel values within the image. This technique is particularly useful for several reasons:

Improved Data Consistency: Since the coin images are all different in color, normalization ensures that the pixel values in an image have consistent and standardized scales.

Reduced Sensitivity to Intensity Variations: The coin images have variations in illumination and contrast due to factors like lighting conditions. Normalization can help mitigate these variations by scaling the pixel values, making the image more robust to such changes.

Enhanced Visualization: Normalized images often have better visual quality, which can aid in the human interpretation of images and make them more suitable for display or presentation purposes.

Although there are several methods of normalization, the one used is Min-Max Scaling.

In [290]:
image_path = r'C:\Users\User\Downloads\Desafio2 - contador de moedas-20231006\Challenge2\coins102.jpg'

# Load the image
img = cv2.imread(image_path)

img_norm = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U) # normaliza para [0, 255], inteiros

# Display the normalized image
cv2.imshow('Normalized Image', img_norm)

# Wait for a key press and close windows
cv2.waitKey(0)
cv2.destroyAllWindows()

## Sharpen the image

It was testes several convolution methods but the ones that provided the best result after binarizing the image was sharpening the image with cv2.filter2D(img_norm, -1, sharpenkernel) and then applying a blur. 

In [305]:
sharpenkernel = np.array((
[0, -1, 0],
[-1,  5, -1],
[0, -1, 0]), dtype=np.int8)

img_sharpened2 = cv2.filter2D(img_norm, -1, sharpenkernel)


cv2.imshow("Image sharpened", img_sharpened2)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Blur the image

Blur is a technique used to reduce the level of detail or sharpness in an image. It has both practical and creative applications, but the main reason they were used was to:

Noise Reduction: since applying a blur filter can help smooth out these variations and make the image appear cleaner.

Image Smoothing: since blurring can be used to smooth or soften the transitions between different regions or objects in an image and help to reduce the appearance of fine details or imperfections, creating a more aesthetically pleasing result.

While doing the A/B testing it was tried to enhance the brightness of the image, but that obviouly didn't work because after binarizing the image it seemed that the coins had more holes, what could cause difficulties when trying to detect the edges of the coins. 

Blurring the image was then the approach chosen. cv2.blur(img, (5,5)) and cv2.GaussianBlur(img, (5, 5)) were tested but the second one proved to be the best for the kind of images provided.

In [306]:
# Apply Gaussian Blur
blurred_img = cv2.GaussianBlur(img_sharpened2, (7, 7), 2)

# Display the blurred image
cv2.imshow('Blurred Image', blurred_img)

# Wait for a key press and close windows
cv2.waitKey(0)
cv2.destroyAllWindows()


## Gray out the image

In [297]:
# Convert the image to grayscale
gray_img = cv2.cvtColor(img_sharpened2, cv2.COLOR_BGR2GRAY)

# Apply Gaussian Blur
blurred_img = cv2.GaussianBlur(gray_img, (7, 7), 2)

cv2.imshow('Blurred Image', blurred_img)
# Wait for a key press and close windows
cv2.waitKey(0)
cv2.destroyAllWindows()


## Binarize the images

In [298]:

# Binarize using Otsu's thresholding
t, imgBin = cv2.threshold(blurred_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Display the binarized image
cv2.imshow('Binarized Image', imgBin)

# Wait for a key press and close windows
cv2.waitKey(0)
cv2.destroyAllWindows()

  ## Apply morphological operations (Closing)
  
The approach used to better separate the coins and to deal with the holes in the coins was to apply closing. Using the imgClosed = cv2.morphologyEx(imgBin, cv2.MORPH_CLOSE, strel) didn't work very well to hide the holes in the image, so it was decided to apply closing by applying Dilatation followed by Erosion.

### Apply Dilation

Dilation "grows" or expands the white regions in the binary image. It fills in small gaps, connects nearby white pixels, and enlarges objects. The size and shape of the structuring element determine how much dilation occurs. A larger structuring element produces a more significant dilation effect. The arguments in the cv2.getStructuringElement() and cv2.dilate() should be used careful to achieve a good dilatation result. 

After many tries, the arguments below are the ones that provided an optimal result.

In [320]:
strel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

In [321]:
# Display the dilated image
imgDil = cv2.dilate(imgBin, strel, iterations=8)
cv2.imshow("Dilated Image", imgDil)
# Wait for a key press and close windows
cv2.waitKey(0)
cv2.destroyAllWindows()

### Apply Erosion

After performing dilation, the resulting image is subjected to an erosion operation.
Similar to dilation, the kernel is moved over the image, but this time, the output pixel becomes white only if all the pixels under the kernel are white.
Erosion "shrinks" or erodes the white regions in the binary image. It removes small protrusions, reduces object sizes, and smoothens object boundaries.
Like with dilation, the size and shape of the structuring element influence the extent of erosion. A larger structuring element produces less erosion effect.

In [322]:
strel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

In [323]:
# Apply morphological operation: Erosion
imgEro = cv2.erode(imgDil, strel, iterations = 9)

# Display the eroded image
cv2.imshow("Eroded Image", imgEro)


# Wait for a key press and close windows
cv2.waitKey(0)
cv2.destroyAllWindows()

## Count the number of coins

In [324]:

# Find contours
contours, _ = cv2.findContours(imgEro, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

area = {}
for i in range(len(contours)):
    cnt = contours[i]
    ar = cv2.contourArea(cnt)
    area[i] = ar

srt = sorted(area.items(), key=lambda x: x[1], reverse=True)
results = np.array(srt).astype("int")
num = np.argwhere(results[:, 1] > 500).shape[0]

for i in range(1, num):
    img = cv2.drawContours(img, contours, results[i, 0], (0, 255, 0), 3)

print("Number of coins is", num - 1)
cv2.imshow("Final", img)
#cv2.imshow("Eroded Image", imgEro)

# Wait for a key press to continue to the next image
cv2.waitKey(0)
cv2.destroyAllWindows()


Number of coins is 93


## Repeat all the steps for all the coins

To apply the algorithm in a chosen images, copy the path to the image where the images are located and paste in the 'directory' variable defined below. The images should be in 'jpg'. If your images are in a diferent format, you can change the format in the  if filename.endswith(".format") to the desired format.

In [332]:
import re
import os
import cv2
import numpy as np

directory = r'C:\Users\User\Downloads\Desafio2 - contador de moedas-20231006\Challenge2'

for filename in os.listdir(directory):
    if filename.endswith(".jpg"):
        image_path = os.path.join(directory, filename)

        # Load the image
        img = cv2.imread(image_path)
        
        # Normalize the image 
        img_norm = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U) # normaliza para [0, 255], inteiros
        
        #Sharpen the image
        sharpenkernel = np.array((
        [0, -1, 0],
        [-1,  5, -1],
        [0, -1, 0]), dtype=np.int8)

        img_sharpened2 = cv2.filter2D(img_norm, -1, sharpenkernel)

        # Apply Gaussian Blur
        blurred_img = cv2.GaussianBlur(img_sharpened2, (7, 7), 2) # 2 e 4 são os melhores
        
        # Convert the blurred image to grayscale
        img_gray = cv2.cvtColor(blurred_img, cv2.COLOR_BGR2GRAY)
        
        # Binarize using Otsu's thresholding
        t, imgBin = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Apply rectangular dilation
        strel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        imgDil = cv2.dilate(imgBin, strel, iterations=8)
        
        # Apply morphological operation: Erosion
        strel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        imgEro = cv2.erode(imgDil, strel, iterations=9)
        
        # Find contours
        contours, _ = cv2.findContours(imgEro, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

        area = {}
        for i in range(len(contours)):
            cnt = contours[i]
            ar = cv2.contourArea(cnt)
            area[i] = ar

        srt = sorted(area.items(), key=lambda x: x[1], reverse=True)
        results = np.array(srt).astype("int")
        num = np.argwhere(results[:, 1] > 500).shape[0]

        for i in range(1, num):
            img = cv2.drawContours(img, contours, results[i, 0], (0, 255, 0), 3)
        
        match = re.search(r'coins(\d+)\.jpg', filename)
        digit = match.group(1)

        print("Number of coins found is", num - 1, "out of", digit, ":", ((num-1)/int(digit))*100, '%')
        cv2.imshow(filename, img)
        
        # Wait for a key press and close windows
        cv2.waitKey(0)
        cv2.destroyAllWindows()


Number of coins found is 10 out of 10 : 100.0 %
Number of coins found is 91 out of 100 : 91.0 %
Number of coins found is 93 out of 102 : 91.17647058823529 %
Number of coins found is 30 out of 30 : 100.0 %
Number of coins found is 42 out of 50 : 84.0 %
Number of coins found is 6 out of 6 : 100.0 %
